In [4]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

ERROR: Could not find a version that satisfies the requirement sklearn
ERROR: No matching distribution found for sklearn


In [5]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [6]:
import pandas as pd
df = pd.read_csv("exoplanet_data.csv")

# Read the CSV and Perform Basic Data Cleaning

In [9]:
# Exploratory data analysis
import numpy as np
from pathlib import Path
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
import matplotlib.pyplot as plt
import math
import time
from  scipy import ndimage
from sklearn.preprocessing import normalize
import scipy


In [ ]:
# Inspect rows and columns with missing data and decide what to do with them

# Drop rows/columns with missing data or you can replace missing data

# Select your features (columns)

In [ ]:
# Set your y to be poi-disposition and your X to be all other columns


In [ ]:
# Drop *unimportant, irrelevant, or duplicate columns


# Create a Train Test Split


# Pre-processing

In [ ]:
# Scale your data using appropriate scaler


# Train various  models



## Model 1

In [ ]:
# Instantiate model 1 (e.g., logistic model)


# Fit model using GridSearchCV if it makes sense


In [ ]:
# Look at model evalauation metrics - there are more options than just score


## Model 2

In [ ]:
# Instantiate model 2 (e.g., random forest model)


# Fit model using GridSearchCV if it makes sense



In [ ]:
# Look at model evalauation metrics - there are more options than just score


## Feel free to test more models

# Find best model and safe

In [ ]:
# Compare metrics and determine best model


In [ ]:
# Save best model - replace "your_model" with your model variable

import joblib
filename = 'exoplanet_model.sav'
joblib.dump(your_model, filename)